### Helps - Coding with LLMs 301


In [1]:
# Ooodles of Imports
import os
import json
from icecream import ic
import typer
from rich.console import Console
from rich import print
from typing import List
from pydantic import BaseModel
from loguru import logger
import pudb
from typing_extensions import Annotated

console = Console()
app = typer.Typer()
from langchain.llms import GPT4All
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from typing import Any, Optional
from langchain.output_parsers.openai_functions import OutputFunctionsParser
from langchain.schema import FunctionMessage


from langchain.schema import (
    Generation,
    OutputParserException,
)
import mercury as mr

In [2]:
# Useful helpers
def model_to_openai_function(cls):
    return {"name": cls.__name__, "parameters": cls.model_json_schema()}


class JsonOutputFunctionsParser2(OutputFunctionsParser):
    """Parse an output as the Json object."""

    def parse_result(self, result: List[Generation]) -> Any:
        function_call_info = super().parse_result(result)
        if self.args_only:
            try:
                # Waiting for this to merge upstream
                return json.loads(function_call_info, strict=False)
            except (json.JSONDecodeError, TypeError) as exc:
                raise OutputParserException(
                    f"Could not parse function call data: {exc}"
                )
        function_call_info["arguments"] = json.loads(function_call_info["arguments"])
        return function_call_info


from IPython.display import display, HTML


def print_line():
    display(HTML("<hr>"))
def print_prompt(prompt):
    print ("Prompt:")
    for m in prompt.messages:
        print(f"{type(m)}  {m.prompt}")

### Langchain - Super cool, we'll use it, but not our focus

### Compilers - Prompts and Execution


In [3]:
# Everyone wants to be a comedian
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

joke_prompt = ChatPromptTemplate.from_template("tell me {count} jokes about {topic}")
print(joke_prompt.messages)

[
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=['count', 'topic'],
            output_parser=None,
            partial_variables={},
            template='tell me {count} jokes about {topic}',
            template_format='f-string',
            validate_template=True
        ),
        additional_kwargs={}
    )
]

In [4]:
# Compile the program and run on a familiar GPU

model = ChatOpenAI()
chain = joke_prompt | model

# Run it => Invoke()

topic = "Software Engineers"
count = 2
result = chain.invoke({"topic": topic, "count": count})
print(result)
print_line()
print(result.content)

AIMessage(
    content='Sure, here are two jokes about software engineers:\n\n1. Why do software engineers prefer 
gardening?\nBecause they love to "debug" the plants!\n\n2. Why did the software engineer go broke?\nBecause he had 
way too many "bugs" in his code and couldn\'t afford the exterminator!',
    additional_kwargs={},
    example=False
)

Sure, here are two jokes about software engineers:

1. Why do software engineers prefer gardening?
Because they love to "debug" the plants!

2. Why did the software engineer go broke?
Because he had way too many "bugs" in his code and couldn't afford the exterminator!

In [5]:
# Compile the program and run on our CPU
local_model = GPT4All(model="./llama2.bin")
local_chain = joke_prompt | model


result = local_chain.invoke({"topic": topic, "count": count})
print(result)
print_line()
print(result)

Found model file at  ./llama2.bin
falcon_model_load: loading model from './llama2.bin' - please wait ...
falcon_model_load: n_vocab   = 65024
falcon_model_load: n_embd    = 4544
falcon_model_load: n_head    = 71
falcon_model_load: n_head_kv = 1
falcon_model_load: n_layer   = 32
falcon_model_load: ftype     = 2
falcon_model_load: qntvr     = 0
falcon_model_load: ggml ctx size = 3872.64 MB
falcon_model_load: memory_size =    32.00 MB, n_mem = 65536
falcon_model_load: .

objc[82384]: Class GGMLMetalClass is implemented in both /Users/idvorkin/homebrew/lib/python3.10/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libreplit-mainline-metal.dylib (0x13f54c208) and /Users/idvorkin/homebrew/lib/python3.10/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libllamamodel-mainline-metal.dylib (0x13f978208). One of the two will be used. Which one is undefined.


....................... done
falcon_model_load: model size =  3872.59 MB / num tensors = 196


AIMessage(
    content="Sure, here are two jokes for you:\n\n1. Why do software engineers prefer dark mode?\nBecause light 
attracts bugs!\n\n2. Why did the software engineer go broke?\nBecause he lost his domain in a bet and couldn't 
afford to buy a new one!",
    additional_kwargs={},
    example=False
)

AIMessage(
    content="Sure, here are two jokes for you:\n\n1. Why do software engineers prefer dark mode?\nBecause light 
attracts bugs!\n\n2. Why did the software engineer go broke?\nBecause he lost his domain in a bet and couldn't 
afford to buy a new one!",
    additional_kwargs={},
    example=False
)

### The Operating System - Functions 

In [6]:
# The rain in spain
# Tell the model the "OS" supports getting the weather


# define a callable function
class GetWeather(BaseModel):
    City: str


get_weather = model_to_openai_function(GetWeather)

weather_prompt_template = "What's the weather in {place}"
model = ChatOpenAI()
weather_prompt = ChatPromptTemplate.from_template(weather_prompt_template)

chain = weather_prompt | model.bind(
    functions=[get_weather] # tell model we can call it.
)  


response = chain.invoke({"place": "Spain"})
print(response)

AttributeError: type object 'GetWeather' has no attribute 'model_json_schema'

In [ ]:
# Woah  - Did you see  the bug?
model = ChatOpenAI(model="gpt-4")

# Do "some more programming"
weather_prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "When an API takes a city, infer an appropritiate city"
        ),
        HumanMessagePromptTemplate.from_template(weather_prompt_template),
    ]
)
chain = weather_prompt | model.bind(functions=[get_weather])
response = chain.invoke({"place": "Spain"})
print("Output")
print(response)

In [ ]:
# Back to our functions

weather_with_data = weather_prompt.copy()

# Update prompt with AI's desire to call a function
weather_with_data.append(response)

# Need to make tomorrow's cut, just stamp this please :)
# Will come back and make a dispatcher and call actual functions

weather_with_data.append(
    FunctionMessage(name="GetWeather", content="5 degrees and rainy")
)

print(weather_with_data)


chain = weather_with_data | model.bind(
    functions=[get_weather]
)  # tell model we can call it.

response = chain.invoke({"place": "Spain"})
print_line()
print(response)

In [ ]:
# Innovate - Why do we seperate view from model?


class Joke(BaseModel):
    setup: str
    punch_line: str
    reason_joke_is_funny: str


class GetJokes(BaseModel):
    count: int
    jokes: List[Joke]


get_jokes = model_to_openai_function(GetJokes)

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me {count} jokes about {topic}")
chain = prompt | model.bind(functions=[get_jokes]) | JsonOutputFunctionsParser2()
print(prompt.messages)
response = chain.invoke({"topic": topic, "count": count})
print("Output")
print(response)

In [ ]:
# Innovate - What's better then doing math with a calculator?


solve_math_with_python = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Write code to solve the users problem. the last line of the python  program should print the answer. Do not use sympy"
        ),
        HumanMessagePromptTemplate.from_template("What is the 217th prime"),
    ]
)


class ExecutePythonCode(BaseModel):
    valid_python: str
    code_explanation: str


model = ChatOpenAI(model="gpt-4").bind(
    function_call={"name": "ExecutePythonCode"},  # tell gpt to use this model
    functions=[model_to_openai_function(ExecutePythonCode)],
)


# JsonOutputFunctionParser2 == PrettyPrintOutput

chain = solve_math_with_python | model | JsonOutputFunctionsParser2()
response = chain.invoke({})

print(response["code_explanation"])
print_line()

valid_python = response["valid_python"]
print(valid_python)

In [ ]:
# Ugh ... are you sure you want to do this??
print_line()
# input("Are you sure you want to run this code??")
exec(valid_python)

### Operating System - Memory

In [ ]:
# Basics of Conversational Memory - # no memory
model = ChatOpenAI().bind(temperature=0)
prompt_template = "Tell me another joke"
last_prompt = ""
for i in range(4):
    print_line()
    prompt = ChatPromptTemplate.from_template(prompt_template)
    print (f"Prompt {i}\n", prompt.messages)
    chain = prompt | model
    response = chain.invoke({})
    
    print(f"Response {i}\n", response.content)
    

In [ ]:
# The memory module!

from langchain.memory import  ChatMessageHistory

memory = ChatMessageHistory()

# memory.save_context({"input":"I like software engineer jokes"} , {"output":"That's odd, but fine"})
memory.add_user_message("I like software engineering jokes") 
memory.add_ai_message("OK!")
print (memory.messages)


In [ ]:
# Basics of Conversational Memory w/Memory

memory = ChatMessageHistory()
memory.add_user_message("I like software engineering jokes") 
memory.add_ai_message("OK!")

# human always says the same thing
human_says = "tell me another joke"
prompt = "" # have it outside loop so we can print it
for i in range(4):
    print_line()

    # build prompt, we'll print the last one ...    
    prompt = ChatPromptTemplate.from_messages(memory.messages)
    prompt.append(human_says) 
    # run the chain 
    chain = prompt | model
    
    response = chain.invoke({})
    print(f"Response {i}\n", response.content)

    # store history
    memory.add_user_message(human_says)
    memory.add_ai_message(response.content)

print_line()
print_line()
print(f"Final Prompt\n", prompt.messages)